In [1]:
import pandas as pd

In [2]:
file_list =['df_voix_7_to_0','df_voix_8','df_voix_9','df_voix_10','df_voix_11','df_voix_12','df_voix_13',
           'df_voix_14','df_voix_15','df_voix_16','df_voix_17','df_voix_18','df_voix_19','df_voix_20',
           'df_voix_21','df_voix_22','df_voix_23','df_voix_24']

def readDataFrame(file_name):
    globals()['excel_'+file_name]=pd.ExcelFile('DataFrame pour les législatives (1er tour de 2017)/'+file_name+'.xlsx')
    globals()[file_name] = pd.read_excel(globals()['excel_'+file_name])
    
    
for file_name in file_list:
    readDataFrame(file_name)

In [3]:
# # df.iterrows -> [DataFrame => Series]
# # for i,row in df_voix_24.iterrows():
# #   print(row['Nom'])

In [4]:
# Liste de Nuances Politiques
# Le Plus Voté (Nom, Prénom)
import random
import itertools

list_nuance=[]
list_nom=[]
list_prenom=[]
for file_name in file_list:
    list_nuance.append(globals()[file_name]['NUANCE_candidat_le_plus_voté'].drop_duplicates().tolist())
    list_nom.append(globals()[file_name]['NOM_candidat_le_plus_voté'].tolist())
    list_prenom.append(globals()[file_name]['PRENOM_candidat_le_plus_voté'].tolist())

# Contained 2nd Array Dimension in List -> Remove 2nd Array, and take the elements in list 
list_nuance = list(itertools.chain.from_iterable(list_nuance))
list_nom = sum(list_nom, [])
list_prenom = sum(list_prenom, [])

# Remove Duplicated elements
dict_nuance = dict.fromkeys(list_nuance)
list_candidat = list(set(zip(list_nom, list_prenom)))
    
# Color for Each Politic Party
for key, value in dict_nuance.items():
    dict_nuance[key]="#%06x" % random.randint(0, 0xFFFFFF)

In [ ]:
# Version 1 (No Cluster)
import folium
from folium import plugins

map = folium.Map(location=[46.91160617052484, 2.153649265809747]
                       , zoom_start=5.5)


def colorByNuanceName(Nom):
    
    return dict_nuance[Nom]


for file_name in file_list:

    globals()[file_name].apply(lambda row: folium.Circle(location=[row.loc['latitude'], row.loc['longitude']],
                                             radius=100,
                                             popup=row.loc['NOM_candidat_le_plus_voté'],
                                             fill=True, 
                                             tooltip="""
                                                     Nom : <b>"""+ str(row['NOM_candidat_le_plus_voté']) +"""</b><br>
                                                     Prénom : <b>"""+ str(row['PRENOM_candidat_le_plus_voté']) +"""</b><br>
                                                     % Voix/Exp : <b>"""+ str(row['% Voix/Exp_le_plus_voté']) +"""</b><br>
                                                     Nuance : <b>"""+ str(row['NUANCE_candidat_le_plus_voté']) + """</b><br>
                                                     
                                                    
                                                     
                                                     """, 
                                             color=colorByNuanceName(row.loc['NUANCE_candidat_le_plus_voté']), 
                                             fill_color=colorByNuanceName(row['NUANCE_candidat_le_plus_voté']),
                                             opacity=0.8,
                                             fill_opacity=0.7)
                                             .add_to(map), axis=1)    

map

In [5]:
# # Version 2 (With Cluster)

import folium
from folium import plugins

map2 = folium.Map(location=[46.91160617052484, 2.153649265809747]
                       , zoom_start=5.5)

marker_cluster = plugins.MarkerCluster().add_to(map2)

def colorByNuanceName(Nom):
    
    return dict_nuance[Nom]


for file_name in file_list:
    index = file_list.index(file_name)

    def listCandidates():
        display = """<tr>
                          <td>
                              """+str(row['Nom'])+ """
                          </td>
                          <td>
                              """+str(row['% Voix/Exp'])+ """
                          </td>                                                    
                      </tr>"""
        for i in range(1,8+index):
            display+="""<tr>
                            <td>
                                """+str(row['Nom.'+str(i)])+ """
                            </td>
                          <td>
                              """+str(row['% Voix/Exp.'+str(i)])+ """
                          </td>                                                      
                        </tr>"""
        return """
                  <br>
                  <h5>Total (%)</h5>
                  <table>"""+ display +"""</table>"""

    
    for i, row in globals()[file_name].iterrows():
        popup = """<h5>Le Plus Voté (%)</h5>
        Nom : <b>"""+ str(row['NOM_candidat_le_plus_voté']) +"""</b><br>
        Prénom : <b>"""+ str(row['PRENOM_candidat_le_plus_voté']) +"""</b><br>
        % Voix/Exp : <b>"""+ str(row['% Voix/Exp_le_plus_voté']) +"""</b><br>
        Nuance : <b>"""+ str(row['NUANCE_candidat_le_plus_voté']) + """</b><br>
        """ + listCandidates()
        
        folium.Circle(location=[row['latitude'], row['longitude']],
                                radius=20,
                                tooltip=popup,
                                color= colorByNuanceName(row['NUANCE_candidat_le_plus_voté']),
                                fill_color=colorByNuanceName(row['NUANCE_candidat_le_plus_voté']),
                                fill=True).add_to(marker_cluster)

map2

KeyboardInterrupt: 

In [6]:
map2.save("map2.html")